<a href="https://colab.research.google.com/github/alscop/ESAA-25-2/blob/main/%EB%8B%A8%EC%9D%BC%EB%AA%A8%EB%8D%B8_LGBM%2C_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm -rf ~/.cache/matplotlib

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 2 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/n

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

train = pd.read_csv('train.csv')

In [5]:
DATE_COL   = '영업일자'
TARGET_COL = '매출수량'
KEY_COLS   = ['영업장명', '메뉴명']

# 첫번째 '_' 기준으로 분리
train[KEY_COLS] = train['영업장명_메뉴명'].str.split('_', n=1, expand=True)

In [6]:
import pandas as pd

# 1. [가장 중요] 날짜 데이터 형식 변환을 제일 먼저 수행해야 합니다.
train['영업일자'] = pd.to_datetime(train['영업일자'])

# 2. 기본 날짜 피처 추출 (이제 .dt 접근자 사용 가능)
train['요일'] = train['영업일자'].dt.dayofweek
train['연도'] = train['영업일자'].dt.year
train['월'] = train['영업일자'].dt.month
train['일'] = train['영업일자'].dt.day
train['주'] = train['영업일자'].dt.isocalendar().week.astype(int)

# 3. 주말 여부 생성
# (질문자님의 의도대로 금(4), 토(5), 일(6)을 주말로 설정)
train['주말여부'] = (train['요일'] >= 4).map({True: '주말', False: '주중'})

# 4. 공휴일 설정 (양력 - 매년 고정)
holis = ['01-01', '03-01', '05-05', '06-06', '08-15', '10-03', '10-09', '12-25']
train['공휴일여부'] = train['영업일자'].dt.strftime('%m-%d').isin(holis).astype(int)

# 5. 음력 공휴일 설정 (설날, 추석 등 - 변동 날짜)
lunar_holidays = [
    '2023-01-21', '2023-01-22', '2023-01-23',
    '2023-09-28', '2023-09-29', '2023-09-30',
    '2024-02-09', '2024-02-10', '2024-02-11',
]
train['음력공휴일여부'] = train['영업일자'].dt.strftime('%Y-%m-%d').isin(lunar_holidays).astype(int)

# 6. 대체 및 기타 공휴일 설정 (선거일 등)
substitute_holidays = [
    '2023-01-24', '2023-05-29', '2023-10-02',
    '2024-02-12', '2024-04-10', '2024-05-06'
]
train['대체공휴일여부'] = train['영업일자'].dt.strftime('%Y-%m-%d').isin(substitute_holidays).astype(int)

# 7. 전체 공휴일 통합 (하나라도 해당하면 1)
train['전체공휴일여부'] = ((train['공휴일여부'] == 1) |
                         (train['음력공휴일여부'] == 1) |
                         (train['대체공휴일여부'] == 1)).astype(int)

# 결과 확인
print(train[['영업일자', '요일', '주말여부', '전체공휴일여부']].head())

        영업일자  요일 주말여부  전체공휴일여부
0 2023-01-01   6   주말        1
1 2023-01-02   0   주중        0
2 2023-01-03   1   주중        0
3 2023-01-04   2   주중        0
4 2023-01-05   3   주중        0


In [7]:
# 1. 데이터를 날짜순으로 정렬 (과거 평균을 구하기 위해 필수)
train = train.sort_values('영업일자')

# 2. 월요일 데이터만 추출해서 '누적 과거 평균' 계산
mon_data = train[train['요일'] == 0].copy()
# expanding().mean()은 현재 행 이전까지의 모든 데이터 평균을 구합니다.
# 단, 현재 날짜의 매출이 포함되지 않게 shift(1)을 해줍니다.
mon_data['past_mon_mean'] = mon_data['매출수량'].expanding().mean().shift(1)

# 3. 원본 데이터에 과거 평균값 합치기
train = pd.merge(train, mon_data[['영업일자', 'past_mon_mean']], on='영업일자', how='left')

# 4. 지수 계산: (현재 매출) / (과거 월요일들의 평균)
# 첫 번째 월요일은 과거 데이터가 없으므로 결측치가 생길 수 있어 fillna(1) 처리
train['mon_event_intensity'] = (train['매출수량'] / train['past_mon_mean']).fillna(1.0)

# 확인
print(train[train['요일']==0][['영업일자', '매출수량', 'past_mon_mean', 'mon_event_intensity']].head())



# 월별 메뉴별 총 판매량 집계부터
train_clean = train[train['매출수량'] >= 0].copy()
monthly_agg = train_clean.groupby(['연도', '월', '영업장명', '메뉴명'])['매출수량'].sum().reset_index()
monthly_agg.rename(columns={'매출수량': '월간_메뉴_총판매량'}, inplace=True)

# 월별 전체 판매량 계산 후 구성비 계산
total_monthly_sales = monthly_agg.groupby(['연도', '월', '영업장명'])['월간_메뉴_총판매량'].transform('sum')
monthly_agg['메뉴_비중'] = monthly_agg['월간_메뉴_총판매량'] / total_monthly_sales

# 월별 판매 순위 생성
# 동일할 경우 'min' 방식으로 매김
monthly_agg['메뉴_판매순위'] = monthly_agg.groupby(['연도', '월', '영업장명'])['월간_메뉴_총판매량'].rank(ascending=False, method='min')

# 전월 대비 구성비의 증감폭
# 메뉴별로 시계열 순서에 따라 이전 행과의 차이 계산
monthly_agg = monthly_agg.sort_values(['영업장명', '메뉴명', '연도', '월'])
monthly_agg['비중_증감폭'] = monthly_agg.groupby(['영업장명', '메뉴명'])['메뉴_비중'].diff().fillna(0)


# 기존 데이터프레임에 결합
# '영업장명'과 '메뉴명'을 키로 각 일자별 데이터에 매칭
train_clean = train[train['매출수량'] >= 0].copy()
df_final = pd.merge(train_clean,
                    monthly_agg[['연도', '월', '영업장명', '메뉴명', '메뉴_비중', '메뉴_판매순위', '비중_증감폭']],
                    on=['연도', '월', '영업장명', '메뉴명'], how='left')


def add_detailed_off_status(df, window=7):
    # 분석의 정확성을 위해 날짜순 정렬
    df = df.sort_values(['영업장명', '영업일자'])
    df['휴점여부'] = 0

    # 영업장별로 루프를 돌며 판단 (해당 영업장이 문 닫으면 모든 메뉴 판매량이 0이 될 것을 가정함)
    for shop in df['영업장명'].unique():
        # 해당 영업장의 일자별 총 매출 수량 계산
        shop_daily = df[df['영업장명'] == shop].groupby('영업일자')['매출수량'].sum().reset_index()

        # window 기간 동안 매출 합계가 0인지 확인
        is_zero = shop_daily['매출수량'] == 0
        rolling_zero = is_zero.rolling(window=window).sum() == window

        # 휴점으로 판단된 날짜들 추출
        off_dates = shop_daily.loc[rolling_zero, '영업일자']

        # 원본 데이터프레임에 반영 - 특정 날짜에 해당 영업장의 모든 메뉴 행을 휴점으로 표시
        for off_date in off_dates:
            # rolling이 끝나는 지점부터 역으로 window만큼 휴점 처리
            start_date = off_date - pd.Timedelta(days=window-1)
            mask = (df['영업장명'] == shop) & (df['영업일자'] >= start_date) & (df['영업일자'] <= off_date)
            df.loc[mask, '휴점여부'] = 1

    return df

# 적용
df_final = add_detailed_off_status(df_final)

          영업일자  매출수량  past_mon_mean  mon_event_intensity
193 2023-01-02    17            NaN             1.000000
194 2023-01-02    17      17.000000             1.000000
195 2023-01-02    17       8.500000             2.000000
196 2023-01-02    17       5.666667             3.000000
197 2023-01-02    17       4.500000             3.777778


In [8]:
# 업장 유형 파생변수 생성(범주형)
flow_stores = ['포레스트릿', '화담숲주막', '화담숲카페', '카페테리아']

train['store_type'] = np.where(
    train['영업장명'].isin(flow_stores),
    'flow',      # 유동형
    'normal'     # 일반 식음
)

# 타깃 변환
train['매출수량_log1p'] = np.log1p(train['매출수량'])

# 메뉴 활동성
menu_sell_rate = (
    train.groupby(['영업장명','메뉴명'])['매출수량']
         .apply(lambda x: (x > 0).mean())
         .reset_index(name='sell_rate')
)

train = train.merge(menu_sell_rate, on=['영업장명','메뉴명'], how='left')

# 메뉴X업장 기준 lag 변수
group_cols = ['영업장명', '메뉴명']
target = '매출수량'

for lag in [1, 7, 14]:
    train[f'lag_{lag}'] = (
        train
        .groupby(group_cols)[target]
        .shift(lag)
    )

# 단기 노이즈 제거 + 추세 반영
# Rolling 평균
for win in [3, 7, 14]:
    train[f'roll_{win}_mean'] = (
        train
        .groupby(group_cols)[target]
        .shift(1)
        .rolling(win)
        .mean()
    )

# Rolling 표준편차
train['roll_7_std'] = (
    train
    .groupby(group_cols)[target]
    .shift(1)
    .rolling(7)
    .std()
)

# 스파이크 감지(지난주 대비 증감)
train['diff_1'] = train['lag_1'] - train['lag_7']

/usr/local/lib/python3.12/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.12/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [9]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import LabelEncoder

def preprocess_menu_data(train: pd.DataFrame):
    train = train.copy()

    # ---------------------------------------------------------
    # 메뉴 카테고리화
    def categorize_menu(name):
        name = str(name).lower().replace(" ", "")

        categories = {
            '행사/단체': ['단체', '연회', 'conference', 'ballroom', 'convention', 'opus', 'hall', 'platter', '패키지', '세트', 'openfood', '오픈푸드', '무제한', '뷔페', '정식'],
            '대여/공간': ['대여', '이용료', '렌탈', '의자', '룸', '잔디그늘집'],
            '주류': ['소주', '맥주', '와인', '막걸리', '하이볼', '칵테일', '카스', '테라', '참이슬', '처음처럼', '하이네켄', '버드와이저', '스텔라', '복분자', 'beer', 'wine', 'gls', 'bottle', '샷'],
            '음료/카페': ['아메리카노', '라떼', '에이드', '주스', '식혜', '차', '커피', '티', '콜라', '사이다', '스프라이트', '미숫가루', '생수', '토닉', 'coffee'],
            '구이/BBQ': ['bbq', '삼겹', '구이', '갈비', '목살', '양갈비', '한우', '등심', '스테이크', 'aus', '돈육', '소세지', '킬바사'],
            '면류': ['파스타', '스파게티', '짜장', '짬뽕', '우동', '국수', '냉면', '라면', '소바', '알리오', '까르보나라', '메밀'],
            '국/탕/찌개': ['찌개', '탕', '국밥', '전골', '해장국', '설렁탕', '갱시기', '육개장', '미역국'],
            '분식/튀김': ['떡볶이', '튀김', '돈까스', '핫도그', '치킨', '너겟', '어묵', '순대'],
            '밥류/한식': ['비빔밥', '덮밥', '볶음밥', '리조또', '공깃밥', '햇반', '주먹밥', '김치', '된장', '반찬'],
            '디저트': ['아이스크림', '케이크', '쿠키', '빵', '와플', '빙수', '디저트', '푸딩'],
            '소모품/기타': ['컵', '종이컵', '수저', '젓가락', '접시', '일회용', '가위', '집게', '부탄가스', '쌈장', '소스', '사리', '야채추가']
        }

        for cat, keywords in categories.items():
            for kw in keywords:
                if kw in name:
                    return cat
        return '기타'

    train['메뉴_카테고리'] = train['메뉴명'].apply(categorize_menu)
    return train

def encode_features_train_only(train: pd.DataFrame):
    train = train.copy()

    le_store = LabelEncoder()
    le_menu = LabelEncoder()
    le_cat = LabelEncoder()

    # Train 데이터에 대해서만 fit_transform 수행
    train['영업장명_le'] = le_store.fit_transform(train['영업장명'].astype(str))
    train['메뉴명_le'] = le_menu.fit_transform(train['메뉴명'].astype(str))
    train['카테고리_le'] = le_cat.fit_transform(train['메뉴_카테고리'].astype(str))

    return train, le_store, le_menu, le_cat


# 전처리 수행
train = preprocess_menu_data(train)

# 인코딩
train_encoded, le_store, le_menu, le_cat = encode_features_train_only(train)

In [10]:
# 결과
print(train_encoded[['영업장명_메뉴명', '영업장명', '메뉴명', '메뉴_카테고리', '카테고리_le']].head())

             영업장명_메뉴명        영업장명            메뉴명 메뉴_카테고리  카테고리_le
0  느티나무 셀프BBQ_1인 수저세트  느티나무 셀프BBQ        1인 수저세트   행사/단체       11
1   연회장_Conference M8         연회장  Conference M8   행사/단체       11
2        미라시아_핑크레몬에이드        미라시아        핑크레몬에이드   음료/카페        9
3  라그로타_미션 서드 카베르네 쉬라        라그로타  미션 서드 카베르네 쉬라      기타        2
4          화담숲주막_병천순대       화담숲주막           병천순대   분식/튀김        7


## 제미나이가 통합본 정리해준 전처리 코드들

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.preprocessing import LabelEncoder

# =========================================================
# 1. 함수 정의 (전처리용 헬퍼 함수들)
# =========================================================

def categorize_menu(name):
    """메뉴명을 기반으로 카테고리를 분류하는 함수"""
    name = str(name).lower().replace(" ", "")

    categories = {
        '행사/단체': ['단체', '연회', 'conference', 'ballroom', 'convention', 'opus', 'hall', 'platter', '패키지', '세트', 'openfood', '오픈푸드', '무제한', '뷔페', '정식'],
        '대여/공간': ['대여', '이용료', '렌탈', '의자', '룸', '잔디그늘집'],
        '주류': ['소주', '맥주', '와인', '막걸리', '하이볼', '칵테일', '카스', '테라', '참이슬', '처음처럼', '하이네켄', '버드와이저', '스텔라', '복분자', 'beer', 'wine', 'gls', 'bottle', '샷'],
        '음료/카페': ['아메리카노', '라떼', '에이드', '주스', '식혜', '차', '커피', '티', '콜라', '사이다', '스프라이트', '미숫가루', '생수', '토닉', 'coffee'],
        '구이/BBQ': ['bbq', '삼겹', '구이', '갈비', '목살', '양갈비', '한우', '등심', '스테이크', 'aus', '돈육', '소세지', '킬바사'],
        '면류': ['파스타', '스파게티', '짜장', '짬뽕', '우동', '국수', '냉면', '라면', '소바', '알리오', '까르보나라', '메밀'],
        '국/탕/찌개': ['찌개', '탕', '국밥', '전골', '해장국', '설렁탕', '갱시기', '육개장', '미역국'],
        '분식/튀김': ['떡볶이', '튀김', '돈까스', '핫도그', '치킨', '너겟', '어묵', '순대'],
        '밥류/한식': ['비빔밥', '덮밥', '볶음밥', '리조또', '공깃밥', '햇반', '주먹밥', '김치', '된장', '반찬'],
        '디저트': ['아이스크림', '케이크', '쿠키', '빵', '와플', '빙수', '디저트', '푸딩'],
        '소모품/기타': ['컵', '종이컵', '수저', '젓가락', '접시', '일회용', '가위', '집게', '부탄가스', '쌈장', '소스', '사리', '야채추가']
    }

    for cat, keywords in categories.items():
        for kw in keywords:
            if kw in name:
                return cat
    return '기타'

def add_detailed_off_status(df, window=7):
    """7일 연속 매출 0인 경우 휴점으로 판단"""
    df = df.copy()
    # 날짜순 정렬 필수
    df = df.sort_values(['영업장명', '영업일자'])
    df['휴점여부'] = 0

    for shop in df['영업장명'].unique():
        shop_mask = df['영업장명'] == shop
        shop_daily = df[shop_mask].groupby('영업일자')['매출수량'].sum().reset_index()

        is_zero = shop_daily['매출수량'] == 0
        rolling_zero = is_zero.rolling(window=window).sum() == window
        off_dates = shop_daily.loc[rolling_zero, '영업일자']

        for off_date in off_dates:
            start_date = off_date - pd.Timedelta(days=window-1)
            # 해당 기간 마스킹
            mask = (df['영업장명'] == shop) & (df['영업일자'] >= start_date) & (df['영업일자'] <= off_date)
            df.loc[mask, '휴점여부'] = 1

    return df

def encode_features_train_only(train_df):
    """라벨 인코딩 수행"""
    df = train_df.copy()
    le_store = LabelEncoder()
    le_menu = LabelEncoder()
    le_cat = LabelEncoder()

    df['영업장명_le'] = le_store.fit_transform(df['영업장명'].astype(str))
    df['메뉴명_le'] = le_menu.fit_transform(df['메뉴명'].astype(str))
    df['카테고리_le'] = le_cat.fit_transform(df['메뉴_카테고리'].astype(str))

    return df, le_store, le_menu, le_cat

# =========================================================
# 2. 메인 전처리 실행
# =========================================================

# 1) 데이터 로드
train = pd.read_csv('train.csv')

# 2) 텍스트 정제 및 업장/메뉴명 분리
train['영업장명_메뉴명'] = train['영업장명_메뉴명'].astype(str).str.replace(r"\ufeff|\u200b", "", regex=True)
train['영업장명_메뉴명'] = train['영업장명_메뉴명'].str.replace("\xa0", " ", regex=False).str.strip()
split_data = train['영업장명_메뉴명'].str.split('_', n=1, expand=True)
train['영업장명'] = split_data[0].str.strip()
train['메뉴명'] = split_data[1].str.strip() if split_data.shape[1] > 1 else "미상"

# 3) 날짜 형식 변환 및 기본 날짜 피처
train['영업일자'] = pd.to_datetime(train['영업일자'])
train['연도'] = train['영업일자'].dt.year
train['월'] = train['영업일자'].dt.month
train['일'] = train['영업일자'].dt.day
train['요일'] = train['영업일자'].dt.dayofweek
train['주'] = train['영업일자'].dt.isocalendar().week.astype(int)

# 4) 공휴일 및 주말 정보 생성
train['주말여부'] = (train['요일'] >= 4).map({True: '주말', False: '주중'})

holis = ['01-01', '03-01', '05-05', '06-06', '08-15', '10-03', '10-09', '12-25']
lunar_holidays = ['2023-01-21', '2023-01-22', '2023-01-23', '2023-09-28', '2023-09-29', '2023-09-30', '2024-02-09', '2024-02-10', '2024-02-11']
substitute_holidays = ['2023-01-24', '2023-05-29', '2023-10-02', '2024-02-12', '2024-04-10', '2024-05-06']

train['공휴일여부'] = train['영업일자'].dt.strftime('%m-%d').isin(holis).astype(int)
train['음력공휴일여부'] = train['영업일자'].dt.strftime('%Y-%m-%d').isin(lunar_holidays).astype(int)
train['대체공휴일여부'] = train['영업일자'].dt.strftime('%Y-%m-%d').isin(substitute_holidays).astype(int)
train['전체공휴일여부'] = ((train['공휴일여부'] == 1) | (train['음력공휴일여부'] == 1) | (train['대체공휴일여부'] == 1)).astype(int)

# 5) 정렬 (시계열 연산을 위해 필수)
train = train.sort_values(['영업장명', '메뉴명', '영업일자'])

# 6) 월요일 과거 평균 (Expanding Mean)
mon_data = train[train['요일'] == 0].copy()
mon_data['past_mon_mean'] = mon_data.groupby(['영업장명', '메뉴명'])['매출수량'].expanding().mean().shift(1).reset_index(level=[0,1], drop=True)
train = pd.merge(train, mon_data[['영업일자', '영업장명', '메뉴명', 'past_mon_mean']], on=['영업일자', '영업장명', '메뉴명'], how='left')
train['mon_event_intensity'] = (train['매출수량'] / train['past_mon_mean']).fillna(1.0)

# 7) 월별 집계 통계 (비중, 순위 등)
# 주의: 원본 코드의 로직을 유지하되, train 데이터프레임에 병합
monthly_agg = train.groupby(['연도', '월', '영업장명', '메뉴명'])['매출수량'].sum().reset_index()
monthly_agg.rename(columns={'매출수량': '월간_메뉴_총판매량'}, inplace=True)
total_monthly_sales = monthly_agg.groupby(['연도', '월', '영업장명'])['월간_메뉴_총판매량'].transform('sum')
monthly_agg['메뉴_비중'] = monthly_agg['월간_메뉴_총판매량'] / (total_monthly_sales + 1e-9) # 0나누기 방지
monthly_agg['메뉴_판매순위'] = monthly_agg.groupby(['연도', '월', '영업장명'])['월간_메뉴_총판매량'].rank(ascending=False, method='min')
monthly_agg = monthly_agg.sort_values(['영업장명', '메뉴명', '연도', '월'])
monthly_agg['비중_증감폭'] = monthly_agg.groupby(['영업장명', '메뉴명'])['메뉴_비중'].diff().fillna(0)

train = pd.merge(train, monthly_agg[['연도', '월', '영업장명', '메뉴명', '메뉴_비중', '메뉴_판매순위', '비중_증감폭']],
                 on=['연도', '월', '영업장명', '메뉴명'], how='left')

# 8) 휴점 여부 및 업장 타입
train = add_detailed_off_status(train)
flow_stores = ['포레스트릿', '화담숲주막', '화담숲카페', '카페테리아']
train['store_type'] = np.where(train['영업장명'].isin(flow_stores), 'flow', 'normal')

# 9) 메뉴 카테고리 및 활동성 지표
train['메뉴_카테고리'] = train['메뉴명'].apply(categorize_menu)
train['매출수량_log1p'] = np.log1p(train['매출수량'])

menu_sell_rate = train.groupby(['영업장명','메뉴명'])['매출수량'].apply(lambda x: (x > 0).mean()).reset_index(name='sell_rate')
train = train.merge(menu_sell_rate, on=['영업장명','메뉴명'], how='left')

# 10) 시계열 Lag 및 Rolling 변수 (다시 정렬 후 수행)
train = train.sort_values(['영업장명', '메뉴명', '영업일자'])
group_cols = ['영업장명', '메뉴명']
target = '매출수량'

for lag in [1, 7, 14]:
    train[f'lag_{lag}'] = train.groupby(group_cols)[target].shift(lag)

for win in [3, 7, 14]:
    train[f'roll_{win}_mean'] = train.groupby(group_cols)[target].shift(1).rolling(win).mean()

train['roll_7_std'] = train.groupby(group_cols)[target].shift(1).rolling(7).std()
train['diff_1'] = train['lag_1'] - train['lag_7']

# 11) 결측치 처리 (Lag로 인한 NaN 채우기)
train = train.fillna(0)

# 12) 라벨 인코딩 (최종)
train_encoded, le_store, le_menu, le_cat = encode_features_train_only(train)

# 결과 확인
print("전처리 완료! 데이터 크기:", train_encoded.shape)
print(train_encoded[['영업일자', '영업장명', '메뉴명', '휴점여부', '매출수량_log1p', '카테고리_le']].head())

/usr/local/lib/python3.12/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.12/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


전처리 완료! 데이터 크기: (102676, 36)
         영업일자        영업장명      메뉴명  휴점여부  매출수량_log1p  카테고리_le
0  2023-01-01  느티나무 셀프BBQ  1인 수저세트     0         0.0       11
23 2023-01-02  느티나무 셀프BBQ  1인 수저세트     0         0.0       11
46 2023-01-03  느티나무 셀프BBQ  1인 수저세트     0         0.0       11
69 2023-01-04  느티나무 셀프BBQ  1인 수저세트     0         0.0       11
92 2023-01-05  느티나무 셀프BBQ  1인 수저세트     0         0.0       11


In [12]:
train_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 102676 entries, 0 to 102675
Data columns (total 36 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   영업일자                 102676 non-null  datetime64[ns]
 1   영업장명_메뉴명             102676 non-null  object        
 2   매출수량                 102676 non-null  int64         
 3   영업장명                 102676 non-null  object        
 4   메뉴명                  102676 non-null  object        
 5   연도                   102676 non-null  int32         
 6   월                    102676 non-null  int32         
 7   일                    102676 non-null  int32         
 8   요일                   102676 non-null  int32         
 9   주                    102676 non-null  int64         
 10  주말여부                 102676 non-null  object        
 11  공휴일여부                102676 non-null  int64         
 12  음력공휴일여부              102676 non-null  int64         
 13  대체공휴일여부            

# 단일모델 모델링
LGBM, XGBoost + CatBoost(?)

## LGBM

##XGBoost